<a href="https://colab.research.google.com/github/aaubs/ds-master/blob/main/notebooks/M2-topicmodel-openalex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Installing Gensim and PyLDAvis
!pip install -qq -U gensim
!pip install -qq pyLDAvis
!pip install -qq --upgrade numpy

In [ ]:
import pandas as pd
import numpy as np
import tqdm #progress bar
import requests
import json
import math

import spacy #spacy for quick language prepro
nlp = spacy.load('en_core_web_sm') #instantiating English module

# sampling, splitting
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split


# loading ML libraries
from sklearn.pipeline import make_pipeline #pipeline creation
from sklearn.feature_extraction.text import TfidfVectorizer #transforms text to sparse matrix
from sklearn.linear_model import LogisticRegression #Logit model
from sklearn.metrics import classification_report #that's self explanatory
from sklearn.decomposition import TruncatedSVD #dimensionality reduction
from xgboost import XGBClassifier

import altair as alt #viz

# topic modeling

from gensim.corpora.dictionary import Dictionary # Import the dictionary builder
from gensim.models import LdaMulticore # we'll use the faster multicore version of LDA


# Import pyLDAvis
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

%matplotlib inline
pyLDAvis.enable_notebook()

## Data Loading

In [ ]:
# Load remote file - dataframe of 1072 publications records on NLP research from Openalex
data = pd.read_csv('https://raw.githubusercontent.com/AI-Growth-Lab/SciNerTopic/main/data/nlp_openalex.csv')

In [ ]:
#@title Load Data from OpenAlex

#@markdown You can check out the list of concepts witht heir IDs [here](https://docs.google.com/spreadsheets/d/1LBFHjPt4rj_9r0t0TTAlT68NwOtNH8Z21lBMsJDMoZg/edit#gid=575855905), e.g., NLP c204321447 
# specify endpoint
endpoint = 'works'
concept = "'C548445021'" #@param {type:"string"}
oa = True #@param {type:"boolean"}
nDocs = 996 #@param {type:"slider", min:200, max:3000, step:1}
from_pub_date = "2015-01-01" #@param {type:"date"}
#@markdown Enter your email for API call to OpenAlex. It is not stored but just used for the API call to OpenAlex.
email = 'roman@business.aau.dk'#@param {type:"string"} 


def OA(oa):
  if True:
    return 'true'
  else:
    return 'false'



oa_str = OA(oa)

# build the 'filter' parameter
filters = ",".join((
    f'concepts.id:{concept}',
    'is_paratext:false', 
    f'from_publication_date:{from_pub_date}',
    f'is_oa:{oa_str}'
))

# put the URL together
filtered_works_url = f'https://api.openalex.org/{endpoint}?mailto={email}&filter={filters}'
print(f'complete URL with filters:\n{filtered_works_url}')


paging_param = 'per-page=100&cursor=*'

works_query = f'{filtered_works_url}&{paging_param}'

response = requests.get(works_query)
meta = json.loads(response.text)['meta']
next_cursor = meta['next_cursor']
results_alx = json.loads(response.text)['results']


cycles = math.floor((meta['count'] - 100) / meta['per_page'])+1
if cycles > 30:
  cycles = int(nDocs/100)

df_input = []

for result in results_alx:
  if result['abstract_inverted_index']:
    abs = ' '.join(result['abstract_inverted_index'].keys())
    df_input.append((result['id'], result['doi'],result['title'],result['publication_year'],abs))

for cycle in range(cycles):
  cycle_query = f'{works_query[:-1]}{next_cursor}'
  response = requests.get(cycle_query)
  meta = json.loads(response.text)['meta']
  next_cursor = meta['next_cursor']
  results_alx = json.loads(response.text)['results']
  for result in results_alx:
    if result['abstract_inverted_index']:
      abs = ' '.join(result['abstract_inverted_index'].keys())
      df_input.append((result['id'], result['doi'],result['title'],result['publication_year'],abs))


data = pd.DataFrame(df_input, columns=['id','doi','title','publication_year','abstract'])

print(f'Downloaded {str(len(data))} documents')

data.head()

In [ ]:
data.shape

In [ ]:
data.columns

In [ ]:
data

In [ ]:
data ['text'] = data['title'] + '. ' + data['abstract']

In [ ]:
# run progress bar and clean up using spacy but without some heavy parts of the pipeline

clean_text = []

pbar = tqdm.tqdm(total=len(data['text']),position=0, leave=True)

for text in nlp.pipe(data['text'], disable=["tagger", "parser", "ner"]):

  txt = [token.lemma_.lower() for token in text 
         if token.is_alpha 
         and not token.is_stop 
         and not token.is_punct]

  clean_text.append(" ".join(txt))

  pbar.update(1)

In [ ]:
data

In [ ]:
data['text_clean'] = clean_text

In [ ]:
# preprocess texts (we need tokens)
tokens = []

for summary in nlp.pipe(data['text_clean'], disable=["ner"]):
  proj_tok = [token.lemma_.lower() for token in summary 
              if token.pos_ in ['NOUN', 'PROPN', 'ADJ', 'ADV'] 
              and not token.is_stop
              and not token.is_punct] 
  tokens.append(proj_tok)

In [ ]:
data['tokens'] = tokens

In [ ]:
# Create a Dictionary from the articles: dictionary
dictionary = Dictionary(data['tokens'])
# filter out low-frequency / high-frequency stuff, also limit the vocabulary to max 1000 words
dictionary.filter_extremes(no_below=5, no_above=0.5, keep_n=1000)
# construct corpus using this dictionary
corpus = [dictionary.doc2bow(doc) for doc in data['tokens']]

In [ ]:
# Training the model
lda_model = LdaMulticore(corpus, id2word=dictionary, num_topics=5, workers = 4, passes=10)

In [ ]:
# Let's try to visualize
lda_display = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)

In [ ]:
 # Let's Visualize
pyLDAvis.display(lda_display)

In [ ]:
from gensim.models.coherencemodel import CoherenceModel

In [ ]:
cm = CoherenceModel(model=lda_model, texts = data['tokens'], coherence='c_v')
coherence = cm.get_coherence()  # get coherence value

In [ ]:
coherence